In [11]:
import pandas as pd
from sqlalchemy import create_engine
import io
import time


In [ ]:
# 数据库连接配置
DB_USER = 'ecommerce_user'
DB_PASSWORD = 'password'
DB_HOST = '127.0.0.1'
DB_PORT = '5432'
DB_NAME = 'ecommerce_db'
DATABASE_URI = f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
path_base = '../data/'
CHUNK_SIZE = 10000     

In [13]:

def fast_import_with_copy(DATABASE_URI,CSV_FILE_PATH,TABLE_NAME,CHUNK_SIZE):
    # 1. Create the engine and obtain the original connection.
    engine = create_engine(DATABASE_URI)
    
    # Obtain the underlying psycopg2 connection object (raw_connection).
    # Because `copy_expert` is a psycopg2 method, not a standard SQLAlchemy method,
    conn = engine.raw_connection()
    cur = conn.cursor()

    print(f"🚀 Start high-speed import: {CSV_FILE_PATH} (records each time {CHUNK_SIZE} num)...")
    start_time = time.time()
    total_rows = 0
    chunk_count = 0

    try:
        # 2. Read CSV in chunks
        with pd.read_csv(CSV_FILE_PATH, chunksize=CHUNK_SIZE) as reader:
            for chunk in reader:
                chunk_count += 1
                
                # --- Data processing area (optional) ---
                # Here you can use Pandas to modify data.
                # chunk['new_col'] = chunk['price'] * 2
                # -------------------------

                # 3. Convert a DataFrame to in-memory CSV format (StringIO)
                output = io.StringIO()
                chunk.to_csv(
                    output, 
                    sep='\t',       # Use tabs as separators to avoid escaping issues caused by commas.
                    index=False,    # No index included
                    header=False    # Table headers are not included (COPY does not require table headers).
                )
                output.seek(0) # The pointer returns to the head, ready to read.

                # 4. Execute the COPY command
                # STDIN indicates reading from standard input (which is our output memory stream).
                try:
                    cur.copy_expert(
                        f"COPY {TABLE_NAME} FROM STDIN WITH (FORMAT CSV, DELIMITER '\t')",
                        output
                    )
                    conn.commit() # Commit transaction
                except Exception as db_err:
                    conn.rollback() # If an error occurs, rollback.
                    print(f"❌ Database write error: {db_err}")
                    raise

                rows_in_chunk = len(chunk)
                total_rows += rows_in_chunk
                print(f"   -> The {chunk_count} batch of writes has completed ({rows_in_chunk} entries)")

        end_time = time.time()
        print(f"🎉 Import completed quickly! Total rows: {total_rows} time taken: {end_time - start_time:.2f} seconds.")

    except FileNotFoundError:
        print(f"❌ Error: File not found{CSV_FILE_PATH}")
    except Exception as e:
        print(f"❌ An unknown error occurred: {e}")
    finally:
        cur.close()
        conn.close()

In [14]:
list = [ {'table':'olist_customers','csv':'olist_customers_dataset.csv'},{'table':'olist_geolocation','csv':'olist_geolocation_dataset.csv'},
{'table':'olist_order_items','csv':'olist_order_items_dataset.csv'} ,{'table':'olist_order','csv':'olist_orders_dataset.csv'},
{'table':'olist_order_payments','csv':'olist_order_payments_dataset.csv'},{'table':'olist_order_reviews','csv':'olist_order_reviews_dataset.csv'},
{'table':'olist_products','csv':'olist_products_dataset.csv'},{'table':'olist_sellers','csv':'olist_sellers_dataset.csv'},{'table':'product_category_name_translation','csv':'product_category_name_translation.csv'}]
for item in list:
    print(item['table'])
    print(item['csv'])
    CSV_FILE_PATH =path_base+ item['csv']      # csv path 
    TABLE_NAME = item['table']
    fast_import_with_copy(DATABASE_URI,CSV_FILE_PATH,TABLE_NAME,CHUNK_SIZE)

olist_customers
olist_customers_dataset.csv
🚀 Start high-speed import: /home/jason/data/Olist_baxi_ecomme_data/olist_customers_dataset.csv (records each time 10000 num)...
   -> The 1 batch of writes has completed (10000 entries)
   -> The 2 batch of writes has completed (10000 entries)
   -> The 3 batch of writes has completed (10000 entries)
   -> The 4 batch of writes has completed (10000 entries)
   -> The 5 batch of writes has completed (10000 entries)
   -> The 6 batch of writes has completed (10000 entries)
   -> The 7 batch of writes has completed (10000 entries)
   -> The 8 batch of writes has completed (10000 entries)
   -> The 9 batch of writes has completed (10000 entries)
   -> The 10 batch of writes has completed (9441 entries)
🎉 Import completed quickly! Total rows: 99441 time taken: 1.45 seconds.
olist_geolocation
olist_geolocation_dataset.csv
🚀 Start high-speed import: /home/jason/data/Olist_baxi_ecomme_data/olist_geolocation_dataset.csv (records each time 10000 num)..